In [39]:
import numpy as np
import pandas as pd

import nltk

In [40]:
df_train = pd.read_csv("DrugReviewAnalysis/drugsComTest_raw.tsv", sep="\t")
df_test = pd.read_csv("DrugReviewAnalysis/drugsComTest_raw.tsv", sep="\t")

In [41]:
df_train = df_train[df_train["condition"].str.contains("users found this comment helpful.")==False]
df_test = df_test[df_test["condition"].str.contains("users found this comment helpful.")==False]

In [42]:
df_train = df_train.drop(columns="Unnamed: 0")

In [43]:
df_test = df_test.drop(columns="Unnamed: 0")

In [44]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [45]:
pip install autocorrect

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [46]:
from autocorrect import Speller

In [47]:
spell = Speller()

In [48]:
def preprocess(text):
    tokens = word_tokenize(text.lower())

    filtered_tokens = [token for token in tokens if token not in stopwords.words("english")]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    processed_text = " ".join(lemmatized_tokens)
    return processed_text


In [38]:
df_train["condition"] = df_train["condition"].apply(lambda x: spell(x))

In [49]:
df_train["condition"] = df_train["condition"].apply(preprocess)

In [50]:
df_train["condition"].unique()

array(['left ventricular dysfunction', 'adhd', 'birth control',
       'opiate dependence', 'benign prostatic hyperplasia',
       'emergency contraception', 'bipolar disorde', 'epilepsy',
       'migraine prevention', 'depression', "crohn 's disease", 'cough',
       'obesity', 'urinary tract infection', 'ibromyalgia',
       'chronic myelogenous leukemia', 'hiv infection', 'insomnia',
       'rheumatoid arthritis', 'vaginal yeast infection',
       'chlamydia infection', 'hirsutism', 'panic disorde', 'migraine',
       'pain', 'irritable bowel syndrome', 'osteoarthritis',
       'constipation', 'bowel preparation', 'psychosis', 'muscle spasm',
       'hepatitis c', 'overactive bladde', 'diabetes , type 2',
       'asthma , maintenance', 'non-small cell lung cance',
       'schizophrenia', 'dysuria', 'smoking cessation', 'anxiety', 'acne',
       'emale infertility', 'constipation , acute',
       'constipation , drug induced', 'erectile dysfunction',
       'trigeminal neuralgia', 'u

In [51]:
df_train.to_csv("DrugReviewAnalysis/drugsTrain_processed.csv", sep="\t", index=False)

**So far only ran for the `condition` column of the training dataset. Took more than an hour to spellcheck the column with the current code, but using the nltk library runs much faster. Need to figure out how to speed up spell checking to use on the `review` feature of the dataset.